In [225]:
import pandas as pd
import numpy as np
import random
import math
import re
from scipy.special import psi #digamma function
from scipy.special import polygamma #derivative of digamma function
from scipy.special import loggamma
from IPython.display import display
import numpy.random
np.random.seed(1)

{'about', 'through', 'ourselves', 'be', 'mightn', "don't", 'any', 'other', 'hadn', 'too', 'above', 'up', 'her', 'were', 'its', 't', 'does', 'should', 'for', 'theirs', 'me', 'has', 'not', 'than', 'my', 'do', 'during', 'all', 'after', 'we', 'i', 'few', 'wouldn', 'these', 'in', 'under', 'this', 'you', 'won', 'at', 'into', 'them', 'he', 'such', 'needn', 'now', "shan't", 'more', 'll', 'doesn', 'nor', "she's", 'can', 'there', 'herself', 'myself', 'most', 'very', 'will', 'hers', 'below', 'before', 'what', 'some', 'an', 'ain', "doesn't", 'why', 'a', 'ours', 've', 'by', 'weren', "isn't", 'until', 'yours', 'no', "you'll", 'couldn', 'themselves', 'doing', "you've", 'am', "that'll", 'yourselves', 'y', 'over', 'when', 'are', 'our', 'being', 'isn', 'him', 'his', 'she', 'shan', 'had', 'own', 'which', 'if', 'between', 'here', 'of', 'each', 'so', 'who', 'having', 'didn', 'don', "needn't", 'that', 'himself', "should've", "couldn't", "hadn't", "wouldn't", 'again', 'your', "mustn't", 'been', 'they', 'is',

In [226]:
def Generate_Data(num_topic,voc_size,num_doc,xi,alpha,eta):
    num_word=np.random.poisson(xi,num_doc) #number of words in each document
    
    beta=np.zeros([num_topic,voc_size])
    for i in range(num_topic):
        beta[i]=np.random.dirichlet(eta)
    
    theta=np.zeros([num_doc,num_topic])
    for d in range(num_doc):
        theta[d]=np.random.dirichlet(alpha)
    
    doc={}
    for d in range(num_doc):

        W=np.zeros([V,num_word[d]])
        for n in range(num_word[d]):
            z=np.random.multinomial(1,theta[d])
            topic_index=np.where(z==1)[0][0]
            w=np.random.multinomial(1,beta[topic_index,])
            W[:,n]=w
        doc[d]=W
                            
    return doc,beta,theta  

In [227]:
class document:
    def __init__(self,words,words_num,phi,gamma):
        self.words=words #words matrix
        self.words_num=words_num #number of words
        self.phi=phi 
        self.gamma=gamma

In [228]:
def sim_data_matrix(doc,V):
    M=int(len(doc))
    N=np.zeros(M,dtype=np.int)   
    doc_matrix=[]
    for d in range(M):
        N[d]=int(doc[d].shape[1]) #number of words in each document
        gamma=np.zeros(K)
        phi=np.zeros([N[d],K])
        
        doc_matrix.append(document(doc[d],N[d],phi,gamma))
    return doc_matrix,N

In [229]:
#initialize gamma and phi
def Initial_gamma_phi():
    
    for d in range(M):
        for i in range(0,K):
            doc_matrix[d].gamma[i]=alpha[i]+N[d]/K
            for n in range(0,N[d]):
                doc_matrix[d].phi[n][i]=1.0/K

In [230]:
#initialize lambda
def Initial_lambda():
    
    for i in range(K):
       
        for j in range(V):
            N0=0
            for d in range(M):
                N0+=sum(doc_matrix[d].words[j]) #total number of word j occured in all documents
            Lambda[i][j]=eta[j]+N0/K
      

In [231]:
#Update hyperparameter alpha
def Updatealpha():
    for i in range(0,K):
        alpha_old[i]=alpha[i]
        
    g=np.zeros(K)
    h=np.zeros(K)
    z=0
    c=0
    for i in range(0,K):
        h[i]=-M*polygamma(1,alpha_old[i])
        g[i]=M*(psi(sum(alpha_old))-psi(alpha_old[i]))
        for d in range(0,M):
            g[i]+=(psi(doc_matrix[d].gamma[i])-psi(sum(doc_matrix[d].gamma)))
    z=M*polygamma(1,sum(alpha_old))
    sum_gh=0
    sum_h=0
    for i in range(0,K):
        sum_gh+=g[i]/h[i]
        sum_h+=1/h[i]
    c=sum_gh/(1/z+sum_h)        
    
    for i in range(0,K):
        alpha[i]=alpha_old[i]-((g[i]-c)/h[i])

In [232]:
#Update hyperparameter eta
def Updateeta():
    for j in range(V):
        eta_old[j]=eta[j]
    g=np.zeros(V)
    h=np.zeros(V)
    z=0
    c=0
    for j in range(V):
        h[j]=-K*polygamma(1,eta_old[j])
        g[j]=K*(psi(sum(eta_old))-psi(eta_old[j]))
        for i in range(K):
            g[j]+=(psi(Lambda[i][j])-psi(sum(Lambda[i])))
    z=K*polygamma(1,sum(eta_old))
    sum_gh=0
    sum_h=0
    for j in range(V):
        sum_gh+=g[j]/h[j]
        sum_h+=1/h[j]
    c=sum_gh/(1/z+sum_h)        
    
    for j in range(V):
        eta[j]=eta_old[j]-((g[j]-c)/h[j])    

In [233]:
def Updategamma(d):
    
    for i in range(0,K):
        gamma_old[d][i]=doc_matrix[d].gamma[i]
        sum_of_phi=0
        for n in range(0,N[d]):
            sum_of_phi+=doc_matrix[d].phi[n][i]

        doc_matrix[d].gamma[i]=alpha[i]+sum_of_phi 

In [234]:
def Updatephi(d):
    
    for n in range(0,N[d]):
        phi_old[d][n]=doc_matrix[d].phi[n]
        phi_sum=0
        for i in range(K):
            j=np.where(np.array(doc_matrix[d].words[:,n]==1))#find out the n-th word index
            doc_matrix[d].phi[n][i]=math.exp(psi(Lambda[i][j])+psi(doc_matrix[d].gamma[i])-psi(sum(Lambda[i]))-psi(sum(doc_matrix[d].gamma)))
            phi_sum+=doc_matrix[d].phi[n][i]
        #normalize phi[d][n]
        for i in range(0,K):
            doc_matrix[d].phi[n][i]=doc_matrix[d].phi[n][i]/phi_sum

In [235]:
def Updatelambda():
    
    for i in range(K):
        
        for j in range(V):
            Lambda_old[i][j]=Lambda[i][j]
            Lambda[i][j]=eta[j]
            for d in range(M):
                for n in range(N[d]):
                    Lambda[i][j]+=doc_matrix[d].phi[n][i]*doc_matrix[d].words[j][n]
        

In [236]:
#find optimal values of variational parameter gamma and phi for each document and variational parameter lambda
def VariationalInference():
    
    global Lambda,Lambda_old
    print("E-STEP: Variational Inference START")
    judge=0

    for d in range(M):
        if ((abs(doc_matrix[d].gamma-gamma_old[d])>error).any() or (abs(doc_matrix[d].phi-phi_old[d])>error).any()):
            judge=1
    for i in range(K):
        if ((abs(Lambda[i]-Lambda_old[i])>error).any()):
            judge=1
    
    t=0
    while (judge==1):
        t+=1
        print("This is iteration",t,"for updating variational parameters")

        
        for d in range(M):
            Updatephi(d)
            Updategamma(d)
            
        Updatelambda()
        judge=0
        for d in range(M):
            if ((abs(doc_matrix[d].gamma-gamma_old[d])>error).any() or (abs(doc_matrix[d].phi-phi_old[d])>error).any()):
                judge=1
        if ((abs(Lambda-Lambda_old)>error).any()):
            judge=1
    print("E-STEP: Variational Inference END")       

In [237]:
def M_step():
    
    print("M-STEP: Model parameter update START")
    #Update alpha until convergence
    t=1
    print("This is iteration",t,"for updating alpha")
    Updatealpha()
    while ((abs(alpha-alpha_old)>error).any()):
        t+=1
        print("This is iteration",t,"for updating alpha")
        Updatealpha()
    
    #Update eta until convergence
    t=1
    print("This is iteration",t,"for updating eta")
    Updateeta()
    
    while ((abs(eta-eta_old)>error).any()):
        t+=1
        print("This is iteration",t,"for updating eta")
        Updateeta()  
    print("M-STEP: Model parameter update END")       

In [238]:
def UpdateL():
    l=0
    for d in range(0,M):
        l+=loggamma(sum(alpha))-loggamma(sum(doc_matrix[d].gamma))
        for i in range(0,K):
            l+=-loggamma(alpha[i])+loggamma(doc_matrix[d].gamma[i])
            l+=(alpha[i]-doc_matrix[d].gamma[i])*(psi(doc_matrix[d].gamma[i])-psi(sum(doc_matrix[d].gamma)))
            for n in range(0,N[d]):
                l+=doc_matrix[d].phi[n][i]*(psi(doc_matrix[d].gamma[i])-psi(sum(doc_matrix[d].gamma)))
                l+=-doc_matrix[d].phi[n][i]*math.log(doc_matrix[d].phi[n][i])
                for j in range(0,V):
                    l+=doc_matrix[d].phi[n][i]*doc_matrix[d].words[j][n]*(psi(Lambda[i][j])-psi(sum(Lambda[i])))
    for i in range(K):
        l+=loggamma(sum(eta))-loggamma(sum(Lambda[i]))
        for j in range(V):
            l+=-loggamma(eta[j])+loggamma(Lambda[i][j])
            l+=(eta[j]-Lambda[i][j])*(psi(Lambda[i][j])-psi(sum(Lambda[i])))
    return l
    

In [239]:
def process():
    Initial_lambda()
    Initial_gamma_phi()

    for d in range(M):
        Updatephi(d)
        Updategamma(d)
        

    Updatelambda()
    VariationalInference()
    M_step()

In [339]:
np.random.seed(1)
K=2 #number of topics
V=10 #Vocabulary size
m=50 #number of document
xi=7
true_alpha=np.array([1.5,0.5])
#true_alpha=[0.8 for i in range(K)]
true_eta=[2.0 for i in range(V)]
doc,true_beta,true_theta=Generate_Data(K,V,m,xi,true_alpha,true_eta)



In [340]:
doc_matrix,N=sim_data_matrix(doc,V)

 
M=len(doc_matrix) #number of documents


error=0.01
L=0 #lower bound of loglikelihood

#initialize alpha and eta
alpha=np.array([1.0,1.0])
eta=[0.1 for i in range(V)]

Lambda_old=np.zeros([K,V])
Lambda=np.zeros([K,V])
gamma_old={}
phi_old={}
alpha_old=np.zeros(K)
eta_old=np.zeros(V)

for d in range(M):
    gamma_old[d]=np.zeros(K)
    phi_old[d]=np.zeros([N[d],K])

In [341]:
iteration=1
print("------This is iteration",iteration,"------")
Initial_lambda()
Initial_gamma_phi()

for d in range(M):
    Updatephi(d)
    Updategamma(d)
Updatelambda()
VariationalInference()
M_step()

new_L=UpdateL()

print("Lower bound of the log likelihood is",new_L)
while (abs(L-new_L)>error):
    L=new_L
    iteration+=1
    print("------This is iteration",iteration,"------")
    process()
    
    new_L=UpdateL()
    
    print("Lower bound of the log likelihood is",new_L)

------This is iteration 1 ------
E-STEP: Variational Inference START
E-STEP: Variational Inference END
M-STEP: Model parameter update START
This is iteration 1 for updating alpha
This is iteration 2 for updating alpha
This is iteration 3 for updating alpha
This is iteration 4 for updating alpha
This is iteration 5 for updating alpha
This is iteration 6 for updating alpha
This is iteration 1 for updating eta
This is iteration 2 for updating eta
This is iteration 3 for updating eta
This is iteration 4 for updating eta
This is iteration 5 for updating eta
This is iteration 6 for updating eta
This is iteration 7 for updating eta
This is iteration 8 for updating eta
This is iteration 9 for updating eta
This is iteration 10 for updating eta
This is iteration 11 for updating eta
This is iteration 12 for updating eta
This is iteration 13 for updating eta
M-STEP: Model parameter update END
Lower bound of the log likelihood is -790.7227922704856
------This is iteration 2 ------
E-STEP: Variation

Lower bound of the log likelihood is -761.6935278366823
------This is iteration 18 ------
E-STEP: Variational Inference START
E-STEP: Variational Inference END
M-STEP: Model parameter update START
This is iteration 1 for updating alpha
This is iteration 2 for updating alpha
This is iteration 3 for updating alpha
This is iteration 1 for updating eta
This is iteration 2 for updating eta
This is iteration 3 for updating eta
M-STEP: Model parameter update END
Lower bound of the log likelihood is -761.5792457943272
------This is iteration 19 ------
E-STEP: Variational Inference START
E-STEP: Variational Inference END
M-STEP: Model parameter update START
This is iteration 1 for updating alpha
This is iteration 2 for updating alpha
This is iteration 3 for updating alpha
This is iteration 1 for updating eta
This is iteration 2 for updating eta
This is iteration 3 for updating eta
M-STEP: Model parameter update END
Lower bound of the log likelihood is -761.4770506708957
------This is iteration 

Lower bound of the log likelihood is -760.6114005100252
------This is iteration 37 ------
E-STEP: Variational Inference START
E-STEP: Variational Inference END
M-STEP: Model parameter update START
This is iteration 1 for updating alpha
This is iteration 2 for updating alpha
This is iteration 3 for updating alpha
This is iteration 1 for updating eta
This is iteration 2 for updating eta
This is iteration 3 for updating eta
M-STEP: Model parameter update END
Lower bound of the log likelihood is -760.5853612228203
------This is iteration 38 ------
E-STEP: Variational Inference START
E-STEP: Variational Inference END
M-STEP: Model parameter update START
This is iteration 1 for updating alpha
This is iteration 2 for updating alpha
This is iteration 3 for updating alpha
This is iteration 1 for updating eta
This is iteration 2 for updating eta
This is iteration 3 for updating eta
M-STEP: Model parameter update END
Lower bound of the log likelihood is -760.5606998593938
------This is iteration 

Lower bound of the log likelihood is -760.2908577620625
------This is iteration 55 ------
E-STEP: Variational Inference START
E-STEP: Variational Inference END
M-STEP: Model parameter update START
This is iteration 1 for updating alpha
This is iteration 2 for updating alpha
This is iteration 3 for updating alpha
This is iteration 1 for updating eta
This is iteration 2 for updating eta
This is iteration 3 for updating eta
M-STEP: Model parameter update END
Lower bound of the log likelihood is -760.2792289054086
------This is iteration 56 ------
E-STEP: Variational Inference START
E-STEP: Variational Inference END
M-STEP: Model parameter update START
This is iteration 1 for updating alpha
This is iteration 2 for updating alpha
This is iteration 3 for updating alpha
This is iteration 1 for updating eta
This is iteration 2 for updating eta
This is iteration 3 for updating eta
M-STEP: Model parameter update END
Lower bound of the log likelihood is -760.268017364873
------This is iteration 5

In [342]:
alpha

array([202.07861512, 202.07861512])

In [343]:
pd.DataFrame(eta,columns=["eta"]).T

,0,1,2,3,4,5,6,7,8,9
eta,630.1,1680.099999,810.1,1110.1,930.1,1080.1,540.1,1500.099999,510.1,1440.099999


In [344]:
pd.DataFrame(true_eta,columns=["true_eta"]).T


,0,1,2,3,4,5,6,7,8,9
true_eta,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
